In [12]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


----

In [13]:
%%timeit
%%cython

cdef unsigned long long int a = 0 #cdef + C/C++ typing
for i in range(1_000_000):
    a += i

14.5 ms ± 295 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit

a = 0
for i in range(1_000_000):
    a += i

34.8 ms ± 461 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


----